In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('reviews.csv') 
df.head()

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z


In [3]:
df.isnull().sum()

ReviewId           0
RecipeId           0
AuthorId           0
AuthorName         0
Rating             0
Review           214
DateSubmitted      0
DateModified       0
dtype: int64

In [4]:
df = df.dropna(subset=['Review'])
df.isnull().sum()

ReviewId         0
RecipeId         0
AuthorId         0
AuthorName       0
Rating           0
Review           0
DateSubmitted    0
DateModified     0
dtype: int64

In [5]:
df['cleaned_reviews'] = df['Review'].str.lower()

In [6]:
df['cleaned_reviews'].head()

0         better than any you can get at a restaurant!
1    i cut back on the mayo, and made up the differ...
2    i think i did something wrong because i could ...
3    easily the best i have ever had.  juicy flavor...
4                                   an excellent dish.
Name: cleaned_reviews, dtype: object

In [7]:
df['cleaned_reviews'] = df['cleaned_reviews'].str.replace('[^\w\s]','') 
df['cleaned_reviews'].head()

/tmp/ipykernel_2061/1630055088.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_reviews'] = df['cleaned_reviews'].str.replace('[^\w\s]','')


0          better than any you can get at a restaurant
1    i cut back on the mayo and made up the differe...
2    i think i did something wrong because i could ...
3    easily the best i have ever had  juicy flavorf...
4                                    an excellent dish
Name: cleaned_reviews, dtype: object

In [8]:
import nltk
from nltk.corpus import stopwords 

In [9]:
# nltk.download('stopwords')

In [10]:
stop_words = stopwords.words('english')

In [11]:
def remove_sw(text):
    filtered_words = []
    for x in text: 
        if x not in stop_words:
            filtered_words.append(x)
    
    return filtered_words 

In [12]:
df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x: x.split(" "))
df['cleaned_reviews'] = df['cleaned_reviews'].apply(lambda x: remove_sw(x))

In [13]:
df['cleaned_reviews'].head()

0                            [better, get, restaurant]
1    [cut, back, mayo, made, difference, sour, crea...
2    [think, something, wrong, could, taste, cornst...
3    [easily, best, ever, , juicy, flavorful, dry, ...
4                                    [excellent, dish]
Name: cleaned_reviews, dtype: object

In [14]:
import numpy as np 

df['is_positive'] = np.where(df['Rating']<3, 0, 1) 

df[['cleaned_reviews','is_positive']]

,cleaned_reviews,is_positive
0,"[better, get, restaurant]",1
1,"[cut, back, mayo, made, difference, sour, crea...",1
2,"[think, something, wrong, could, taste, cornst...",0
3,"[easily, best, ever, , juicy, flavorful, dry, ...",1
4,"[excellent, dish]",1
...,...,...
1401977,"[disappointed, couldnt, wait, make, husband, b...",0
1401978,"[nothing, drain, dont, heat, liquids, put, mil...",1
1401979,"[good, base, recipe, someone, start, quadruple...",1
1401980,"[thank, much, amazing, recipe, lived, kenai, s...",1


In [15]:
# import urllib.request 

# url = 'https://nlp.stanford.edu/data/glove.6B.zip'
# filename = 'glove.6b.zip'
# urllib.request.urlretrieve(url,filename)

In [16]:
import zipfile 

with zipfile.ZipFile('glove.6b.zip', 'r') as zip: 
    zip.extractall()

In [17]:
word_vectors = dict()

def add_wordvector(dict, filename):
    with open(filename, 'r', encoding='utf8') as f:
        for line in f.readlines():
            line = line.split(' ')

            try:
                if line[0] not in stop_words:
                    dict[line[0]] = np.array(line[1:], dtype=float)
            except:
                continue 


add_wordvector(word_vectors, 'glove.6B.50d.txt')
len(stop_words)

179

In [18]:
len(word_vectors)

399851

In [19]:
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
# nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

In [20]:
def lemmatize_token(text):
    lemmatized_token = [lemmatizer.lemmatize(x) for x in text] 
    useful_token = [x for x in lemmatized_token if x in word_vectors]

    return useful_token 

word_token = df['cleaned_reviews'][0] 

lemmatize_token(word_token)


['better', 'get', 'restaurant']

In [21]:
def text_to_vector(text, token_dict=word_vectors):
    processed_token = lemmatize_token(text)

    vectors = []

    for x in processed_token:
        if x not in word_vectors:
            continue 
        
        vectors.append(token_dict[x]) 

    return np.array(vectors, dtype=float) 
    

In [22]:
def vectorize_review(df):
    label = df['is_positive'].to_numpy().astype(int)

    len_wordvector = []

    for text in df['cleaned_reviews']:
        text_as_vector = text_to_vector(text) 

        if text_as_vector.shape[0] == 0:
            text_as_vector = np.zeros(shape=(1,50)) 

        len_wordvector.append(text_as_vector)

    return len_wordvector, label

In [23]:
train_df = df.sample(frac=1, random_state=1)
train_df.reset_index(drop=True, inplace=True) 

split_index_1 = int(len(train_df) * 0.7)
split_index_2 = int(len(train_df) * 0.85)

train_df, val_df, test_df = train_df[:split_index_1], train_df[split_index_1:split_index_2], train_df[split_index_2:]

len(train_df), len(val_df), len(test_df)

(981237, 210265, 210266)

In [24]:
x_train, y_train = vectorize_review(train_df) 

len(x_train), len(x_train[0]), len(x_train[1])

(981237, 52, 2)

In [25]:
token_len = []

for i in range (len(x_train)):
    token_len.append(len(x_train[i])) 

pd.Series(token_len).describe()


count    981237.000000
mean         25.883489
std          18.500794
min           1.000000
25%          13.000000
50%          22.000000
75%          33.000000
max         604.000000
dtype: float64

In [26]:
# from copy import deepcopy 

# def zero_padding(x, max_token_len=605):
#     x_copy = deepcopy(x)

#     for i, j in enumerate(x):
#         x_token_len = j.shape[0]
#         token_len_diff = max_token_len - x_token_len 

#         pad = np.zeros(shape=(token_len_diff, 50))

#         x_copy[i] = np.concatenate([j, pad]) 

#     return np.array(x_copy).astype(float)

In [27]:
# x_train = zero_padding(x_train)

# x_train.shape

In [28]:
import tensorflow as tf

2022-06-15 20:37:17.451275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 20:37:17.451303: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [29]:
x_train = tf.keras.utils.pad_sequences(x_train, maxlen=605, dtype='float32', padding='post') 

x_train.shape


# tf.keras.utils.pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], dtype='float32', padding='post')

(981237, 605, 50)

In [30]:
len(x_train[0]), len(x_train[1])

(605, 605)

In [31]:
y_train.shape

(981237,)

In [32]:
x_val, y_val = vectorize_review(val_df)
x_val = tf.keras.utils.pad_sequences(x_val, maxlen=605, dtype='float32', padding='post')

x_val.shape, y_val.shape

((210265, 605, 50), (210265,))

In [33]:
x_test, y_test = vectorize_review(test_df)
x_test = tf.keras.utils.pad_sequences(x_test, maxlen=605, dtype='float32', padding='post')

x_test.shape, y_test.shape

((210266, 605, 50), (210266,))

In [37]:
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential 

model = Sequential([])

model.add(layers.Input(shape=(605,50))) 
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 605, 64)           29440     
                                                                 
 dropout_3 (Dropout)         (None, 605, 64)           0         
                                                                 
 lstm_4 (LSTM)               (None, 605, 64)           33024     
                                                                 
 dropout_4 (Dropout)         (None, 605, 64)           0         
                                                                 
 lstm_5 (LSTM)               (None, 605, 64)           33024     
                                                                 
 dropout_5 (Dropout)         (None, 605, 64)           0         
                                                                 
 flatten (Flatten)           (None, 38720)            